# Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [78]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [79]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), download_if_missing=True, data_home='data')
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), download_if_missing=True, data_home='data')

## Vectorización

In [80]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [81]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [82]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [83]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [84]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [85]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [86]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [87]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [88]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [89]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [90]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [91]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [92]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [93]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [94]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


## Modelo de clasificación Naïve Bayes

In [95]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [96]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [97]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


In [98]:
import random
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report

## Parte 1
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


In [99]:
# Se crea una clase que guarda los valores 
    # ID de un documento
    # su vector de similitud de coseno
    # dos vectores ordenados de la similitud con un documento y su respectivo ID.
# La clase tambien tiene una funcion para encontrar la N cantidad de documentos similares (y guardar sus respectivos datos)
class Document():
    def __init__(self, idx):
        self.idx = idx
        self.cossim = cosine_similarity(X_train[self.idx], X_train)
        self.sim = np.sort(self.cossim)[::-1]
        self.docsim = np.argsort(self.cossim)[::-1]
        self.class_type = newsgroups_train.target_names[y_train[self.idx]]
        
    def similar_documents(self, amount):
        print(f"Analizando documento {self.idx}, de categorias [{self.class_type}]")
        print("Los documentos mas similares son: ")
        
        similar_docs = []
                    
        for i in range(1, amount + 1):
            idx = self.docsim[:][0][-1-i]
            sim = self.sim[:][0][-1-i]
            class_type = newsgroups_train.target_names[y_train[idx]]
            print (f"{100 * sim:.2f}% similaridad con el documento {idx} de clases [{class_type}]")
            
            similar_docs.append({
                'idx': idx,
                'sim': sim,
                'class_type': class_type
            })
            
        return similar_docs

In [100]:
# Creamos una lista para guardar 5 documentos aleatorios
documents = []
    
# Creamos una de las clases anteriores para cada uno de esos 5 documentos aleatorios
for i in range(5):
    documents.append(
        Document(random.randint(0, len(newsgroups_train.data) - 1))
    )
    
# Guardamos las similitudes en otra lista
similarities = []
    
for i in range(len(documents)):
    similarities.append(documents[i].similar_documents(5))
    print()

Analizando documento 8491, de categorias [rec.autos]
Los documentos mas similares son: 
33.47% similaridad con el documento 1799 de clases [rec.autos]
32.06% similaridad con el documento 4352 de clases [rec.sport.baseball]
32.03% similaridad con el documento 8013 de clases [rec.autos]
28.96% similaridad con el documento 372 de clases [misc.forsale]
27.12% similaridad con el documento 769 de clases [misc.forsale]

Analizando documento 7389, de categorias [comp.os.ms-windows.misc]
Los documentos mas similares son: 
39.36% similaridad con el documento 2140 de clases [comp.os.ms-windows.misc]
35.29% similaridad con el documento 5414 de clases [comp.os.ms-windows.misc]
33.17% similaridad con el documento 8309 de clases [comp.os.ms-windows.misc]
32.68% similaridad con el documento 8680 de clases [comp.os.ms-windows.misc]
23.29% similaridad con el documento 3514 de clases [talk.politics.guns]

Analizando documento 3524, de categorias [rec.sport.hockey]
Los documentos mas similares son: 
41.58

Basandonos unicamente en las clases de los documentos, se puede ver con claridad que todos los articulos comparten clases en comun, y algunos difieren a lo mucho en 1 o 2 clases.

Para poder verificar el contenido de los mismos, tendriamos que leer los 5 documentos originales y los 5 documentos mas similares a cada uno (es decir, 30 documentos), y alguno de ellos son muy extensos. Vamos a emplear ChatGPT para hacer esta tarea mas sencilla.

Nuestro primer prompt es el siguiente:

> En el siguiente mensaje que te voy a escribir, podras leer un texto. Ese es el texto original. Como respuesta al texto original, solo responde "Entendido". En siguientes mensajes, te voy a mandar otros textos. Como respuesta a cada texto adicional, quiero que me digas qué tanto en común tiene el documento enviado y el documento original (una breve descripción de no más de 1 párrafo), y adicionalmente des un porcentaje de similitud aproximado (sin necesidad de explicar nada)

Y los posteriores prompts son los respectivos textos de los documentos.

In [101]:
doc = 4
# Guardar el documento original en un archivo .txt
with open(f"original.txt", "w") as text:
    print(documents[doc].idx)
    print(len(newsgroups_train.data[documents[doc].idx]))
    text.write(newsgroups_train.data[documents[doc].idx])
        
# Guardar los documentos similares en un archivo .txt
for i in range(len(documents)):
    with open(f"document{i}.txt", "w") as text:
        print(len(newsgroups_train.data[similarities[doc][i]['idx']]))
        text.write(newsgroups_train.data[similarities[doc][i]['idx']])


9179
1093
3311
3010
5265
2414
5198


### Documento 1


Respuesta al documento 1:

> El documento similar discute la valoración de la libertad, sugiriendo que aunque muchas personas consideran que la libertad es valiosa, no es universalmente aceptada como un valor objetivo. Ambos textos tratan sobre la libertad y su valor percibido, pero difieren en sus conclusiones sobre su objetividad.

> Porcentaje de similitud: 70%

Respuesta al documento 2:

> El documento similar aborda la relación entre la ciencia, los valores y la moralidad objetiva, sugiriendo que la ciencia, aunque basada en hechos observables, está influenciada por los valores de los observadores. Además, discute la idea de una moralidad objetiva en el contexto de la existencia de un Dios omnipotente. Ambos textos tratan sobre la objetividad en diferentes contextos (valores y moralidad), pero con enfoques distintos.

> Porcentaje de similitud: 60%

Respuesta al documento 3:

> El documento similar expresa una opinión subjetiva sobre la libertad y discute la falta de consenso sobre su significado, cuestionando cómo un valor puede ser objetivo si su definición no lo es. Ambos textos tratan sobre la libertad y su valoración, pero el documento similar enfatiza más la subjetividad y la variabilidad en la interpretación del término.

> Porcentaje de similitud: 75%

Respuesta al documento 4:

> El documento similar critica el uso de la palabra "objetivo" en relación con "valores" y señala la subjetividad inherente a conceptos como el valor y la moralidad. Sugiere que términos como "objetivo" y "subjetivo" deben ser claramente definidos para evitar malentendidos, concluyendo que no existe una moralidad objetiva. Ambos textos tratan sobre la objetividad en valores y moralidad, pero el documento similar se centra en la precisión terminológica y la subjetividad de estos conceptos.

> Porcentaje de similitud: 65%

Respuesta al documento 5:

> El documento similar analiza la definición de "valores" en diferentes contextos, concluyendo que en el contexto científico, los valores se basan en hechos observables, mientras que en un sistema moral, los valores son principios considerados deseables. Argumenta que la ciencia no se basa en valores morales objetivos y cuestiona la posibilidad de una moralidad objetiva. Ambos textos abordan la relación entre valores, objetividad y moralidad, pero desde ángulos distintos, enfocándose en la precisión terminológica y la naturaleza subjetiva de la moralidad.

> Porcentaje de similitud: 70%

### Documento 2

Respuesta al documento 1:

> El documento similar tiene en común con el documento original el uso de frases en inglés, la estructura de mensaje en un formato informal y la referencia a "DoD #8177 = Technician(Dr. Speed) .NOT. Student". Ambos textos mantienen un tono conversacional e informal, y ambos mencionan un identificador de DoD junto con una posición.

> Porcentaje de similitud: 70%

Respuesta al documento 2:

> El documento similar comparte con el documento original el uso de frases en inglés, la estructura informal del mensaje y la referencia a "DoD #8177 = Technician(Dr. Speed) .NOT. Student". Ambos textos también utilizan un identificador de DoD junto con una posición, manteniendo un tono conversacional e informal.

> Porcentaje de similitud: 60%

Respuesta al documento 3:

> El documento similar comparte con el documento original el uso de frases en inglés y la estructura informal del mensaje. Ambos textos mencionan "DoD #8177 = Technician(Dr. Speed) .NOT. Student" como una firma distintiva. Sin embargo, el contenido específico del mensaje es diferente, enfocándose en un tema técnico relacionado con Plexi-Fairings en lugar de referencias a situaciones o preguntas.

> Porcentaje de similitud: 50%

Respuesta al documento 4:

> El documento similar comparte con el documento original el uso de frases en inglés, la estructura informal del mensaje y la firma distintiva "DoD #8177 = Technician(Dr. Speed) .NOT. Student". Sin embargo, el contenido se centra en películas y sus temas, lo que es diferente del contenido del documento original.

> Porcentaje de similitud: 55%

Respuesta al documento 5:

> El documento similar comparte con el documento original el uso de frases en inglés, la estructura informal del mensaje y la firma distintiva "DoD #8177 = Technician(Dr. Speed) .NOT. Student". Ambos textos mantienen un tono conversacional y mencionan temas relacionados con motocicletas. Sin embargo, el contenido específico es diferente, centrándose en una anécdota sobre motocicletas adaptadas para personas en silla de ruedas.

> Porcentaje de similitud: 65%

### Documento 3

Respuesta al documento 1:

> El documento similar comparte el tema de los problemas de calidad con los productos Emerson, específicamente en equipos electrónicos. Ambos textos mencionan fallas y experiencias negativas con productos de la misma marca. Sin embargo, el documento similar es más detallado y anecdótico, mientras que el original es más conciso y directo.

> Porcentaje de similitud: 60%

Respuesta al documento 2:

> El documento similar sugiere un problema específico con el flyback en un Magnavox, similar al comentario sobre el flyback en el documento original. Ambos textos están centrados en diagnosticar problemas técnicos en televisores de diferentes marcas.

> Porcentaje de similitud: 70%

Respuesta al documento 3:

> El documento similar proporciona una explicación técnica sobre la posible causa del problema en un televisor, específicamente relacionado con la pérdida de vacío del tubo de imagen y los posibles efectos secundarios, como una explosión. Esto complementa la discusión técnica sobre problemas de televisores en el documento original.

> Porcentaje de similitud: 75%

Respuesta al documento 4:

> El documento similar parece tomar un tono humorístico o sarcástico en respuesta a la discusión técnica sobre problemas de televisores. Este contraste con el tono técnico del documento original proporciona una perspectiva diferente sobre el tema discutido.

> Porcentaje de similitud: 50%

Respuesta al documento 5:

> El documento similar comparte una experiencia personal con un televisor de marca Goldstar, que es similar en diseño y componentes electrónicos al modelo mencionado en el documento original. Ambos textos discuten problemas técnicos específicos relacionados con el circuito del televisor y la reparación utilizando esquemas y componentes de repuesto.

> Porcentaje de similitud: 80%

### Documento 4

En este documento tuve que cambiar de ChatGPT a Gemini, ya que ChatGPT se enfrascó en un loop de solo responder "Entendido."

Respuesta al documento 1:

> Both documents discuss a serious topic (movies in the original document, hell and salvation in this document).

> Similarity: 10%

Respuesta al documento 2:

> The document is similar to the original document in that they both discuss cars, but the focus here is on renting a car and the insurance involved, whereas the original document was unspecified.

> Similarity: 50%

Respuesta al documento 3:

> Both documents discuss entertainment (watching movies in the original document, watching a documentary in this document).

> Similarity: 10%

Respuesta al documento 4:

> Both documents discuss using technology for entertainment purposes (watching movies in the original document, capturing monitor output for potentially entertainment purposes in this document).

> Similarity: 20%

Respuesta al documento 5:

> This document is similar to the original document in that they both discuss mechanical issues with playback devices. However, the original document focused on movies, while this one focuses on cassette tapes.

> Similarity: 30%

### Documento 5

Este documento (idx 9054) y sus documentos similares hablan de muertes, asesinatos y suicidios en Israel, motivo por el cual Gemini no quiso responder.

> While I aim to be helpful, Google Search may provide a more detailed and accurate response on this topic.

El documento original habla de un carro con pasajero suicida lleno de gas butano que explotó en el medio de dos buses. El documento mas similar 1 y 2 son dos perspectivas distintas del mismo suceso, mientras que el documento 3 y 5 habla acerca de la diferencia entre matar y asesinar en tiempos de guerra. Por otro lado, el documento 4 habla de varios incidentes ocurridos en el mismo lugar de la explosion del carro con pasajero suicida del documento original.

## Parte 2

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [102]:
clf_mnb = MultinomialNB()

# Parámetros a probar en el grid search
parameters = {
    'alpha': (1, 0.5, 0.1, 1e-2, 1e-3)
}

# Configuración de GridSearchCV
kf_mnb = KFold(n_splits=5, shuffle=True)
grid_search_mnb = GridSearchCV(clf_mnb, parameters, cv=kf_mnb, n_jobs=-1, verbose=1, scoring='f1_macro')

# Entrenamiento y ajuste del modelo
grid_search_mnb.fit(X_train, y_train)

# Predicción y evaluación en conjunto de prueba
y_pred = grid_search_mnb.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"El mejor parametro fue [{grid_search_mnb.best_params_}]")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
              precision    recall  f1-score   support

           0       0.57      0.44      0.50       319
           1       0.65      0.71      0.68       389
           2       0.72      0.52      0.60       394
           3       0.59      0.71      0.65       392
           4       0.72      0.70      0.71       385
           5       0.81      0.74      0.78       395
           6       0.83      0.72      0.77       390
           7       0.75      0.73      0.74       396
           8       0.76      0.73      0.75       398
           9       0.93      0.81      0.87       397
          10       0.60      0.93      0.73       399
          11       0.72      0.75      0.74       396
          12       0.72      0.58      0.64       393
          13       0.82      0.78      0.80       396
          14       0.76      0.79      0.77       394
          15       0.56      0.89      0.68       398
          16       0.

In [104]:
clf_cnb = ComplementNB()

# Parámetros a probar en el grid search
parameters_cnb = {
    'alpha': (1, 0.5, 0.1, 1e-2, 1e-3),
    'norm': (True, False)
}

# Configuración de GridSearchCV
kf_cnb = KFold(n_splits=5, shuffle=True) 
grid_search_cnb = GridSearchCV(clf_cnb, parameters_cnb, cv=kf_cnb, n_jobs=-1, verbose=1, scoring='f1_macro')

# Entrenamiento y ajuste del modelo
grid_search_cnb.fit(X_train, y_train)

# Predicción y evaluación en conjunto de prueba
y_pred2 = grid_search_cnb.predict(X_test)
print(classification_report(y_test, y_pred2))
print(f"El mejor parametro fue [{grid_search_cnb.best_params_}]")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
              precision    recall  f1-score   support

           0       0.33      0.47      0.39       319
           1       0.72      0.72      0.72       389
           2       0.71      0.55      0.62       394
           3       0.64      0.69      0.66       392
           4       0.77      0.71      0.74       385
           5       0.80      0.79      0.79       395
           6       0.74      0.72      0.73       390
           7       0.79      0.74      0.77       396
           8       0.81      0.77      0.79       398
           9       0.90      0.84      0.87       397
          10       0.86      0.95      0.90       399
          11       0.76      0.80      0.78       396
          12       0.71      0.57      0.63       393
          13       0.76      0.79      0.77       396
          14       0.77      0.80      0.79       394
          15       0.57      0.86      0.69       398
          16       0

## Parte 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

In [139]:
X_train_T = X_train.T

(11314, 101631)

In [138]:
# Ejemplo: Obtener las palabras más similares a un conjunto de palabras
words_of_interest = ['car', 'computer', 'baseball', 'space', 'government']

# Obtener índices de las palabras de interés
word_indices = [tfidfvect.vocabulary_.get(word) for word in words_of_interest if tfidfvect.vocabulary_.get(word) is not None]

# Calcular similitud del coseno entre las palabras de interés y todas las demás palabras
similarities = cosine_similarity(X_train[:, word_indices], X_train)

# Obtener las palabras más similares para cada palabra de interés
num_similar_words = 5
similar_words = {}
for i, word in enumerate(words_of_interest):
    similar_indices = similarities[i].argsort()[-num_similar_words-1:-1][::-1]
    similar_words[word] = [idx2word[idx] for idx in similar_indices]

print("Palabras más similares:")
for word, similar_list in similar_words.items():
    print(f"{word}: {similar_list}")

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 5 while Y.shape[1] == 101631